In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import matplotlib as plt

# path to dataset
train_dataset = VisDroneDataset(r"C:\Users\Admin\Downloads\VisDrone2019-DET-train-dev", batch_size=32)
val_dataset = VisDroneDataset(r"C:\Users\Admin\Downloads\VisDrone2019-DET-val", batch_size=32)


class VisDroneDataset(Sequence):
    def __init__(self, dataset_dir, batch_size=32, img_size=(640, 640), shuffle=True):
        self.img_size = img_size
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.img_paths, self.labels = self.load_data(dataset_dir)
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.img_paths))
        if shuffle:
            np.random.shuffle(self.indexes)

    def load_data(self, dataset_dir):
        img_paths = []
        labels = []
        for img_file in os.listdir(os.path.join(dataset_dir, "images")):
            img_paths.append(os.path.join(dataset_dir, "images", img_file))
            # Load corresponding labels (YOLO format: class, x, y, width, height)
            label_file = img_file.replace(".jpg", ".txt")
            label_path = os.path.join(dataset_dir, "annotations", label_file)
            labels.append(np.loadtxt(label_path))
        return img_paths, labels

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_imgs = []
        batch_labels = []
        for i in batch_indexes:
            img = cv2.imread(self.img_paths[i])
            img = cv2.resize(img, self.img_size)
            batch_imgs.append(img / 255.0)  # Normalize images
            batch_labels.append(self.labels[i])
        return np.array(batch_imgs), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


KeyboardInterrupt: 

Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def ConvBlock(filters, kernel_size, strides=1, padding="same"):
    return models.Sequential([
        layers.Conv2D(filters, kernel_size, strides=strides, padding=padding),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

def DroneYOLO(input_shape=(640, 640, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)

    # Backbone (RepVGG)
    x = ConvBlock(64, 3, strides=2)(inputs)
    x = ConvBlock(128, 3, strides=2)(x)
    x = ConvBlock(256, 3, strides=2)(x)
    x = ConvBlock(512, 3, strides=2)(x)

    # Neck (PAFPN)
    p5 = ConvBlock(1024, 3)(x)
    p4 = layers.Add()([layers.UpSampling2D()(p5), x])
    p3 = layers.Add()([layers.UpSampling2D()(p4), ConvBlock(512, 3)(p4)])

    # Detection Head
    detections = []
    for p in [p3, p4, p5]:
        cls = layers.Conv2D(num_classes, 1, activation="sigmoid")(p)
        bbox = layers.Conv2D(4, 1)(p)  # (x, y, w, h)
        detections.append(layers.Concatenate()([bbox, cls]))

    model = models.Model(inputs, detections)
    return model

# Build and compile model
model = DroneYOLO()
model.compile(optimizer='adam', loss='mse')
model.summary()


Training and Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred, num_classes=10):
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return precision, recall, f1

# Load dataset
train_dataset = VisDroneDataset("/path/to/VisDrone/train", batch_size=32)
val_dataset = VisDroneDataset("/path/to/VisDrone/val", batch_size=32)

# Training
history = model.fit(train_dataset, epochs=50, validation_data=val_dataset)

# Evaluate on validation set
y_true, y_pred = [], []
for imgs, labels in val_dataset:
    preds = model.predict(imgs)
    y_true.append(labels)
    y_pred.append(preds)
precision, recall, f1 = compute_metrics(np.concatenate(y_true), np.concatenate(y_pred))
print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")
